## **Fake News Classifier Using Bidirectional LSTM RNN**


In [2]:
# configuring the path of Kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
# API to fetch the dataset from Kaggle
!kaggle competitions download -c fake-news

 71% 33.0M/46.5M [00:00<00:00, 143MB/s] 
100% 46.5M/46.5M [00:00<00:00, 160MB/s]


In [5]:
!ls

fake-news.zip  kaggle.json  sample_data


In [6]:
# extracting the compessed Dataset
from zipfile import ZipFile
dataset = '/content/fake-news.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


## **Read Dataset**

In [7]:
import pandas as pd
df = pd.read_csv('/content/train.csv')

df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
df = df.dropna()

In [11]:
## Get the independent features

X = df.drop('label',axis=1)
y = df['label']

In [12]:
## Check whether dataset is balanced or not
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

## **Import Libraries**

In [13]:
import tensorflow as tf

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [14]:
## Vocabulary Size
voc_size = 5000

In [16]:
messages = X.copy()

In [17]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
messages.reset_index(inplace=True)

In [23]:
## Dataset Preprocessing

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

corpus = []

for i in range(0, len(messages)):
  print(i)
  review = re.sub('[^a-zA-Z]',' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [24]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [26]:
## Converting words to indexes
onehot_repr = [one_hot(words,voc_size) for words in corpus]
print(onehot_repr)

[[3164, 344, 556, 1907, 1838, 1913, 2351, 1184, 2067, 2124], [2700, 4883, 622, 2697, 776, 633, 951], [4710, 2481, 517, 522], [481, 2234, 3503, 1194, 2455, 1652], [1654, 776, 4115, 4109, 1209, 1857, 776, 884, 4227, 2534], [2053, 2726, 2080, 3822, 3163, 1241, 1852, 4319, 1916, 755, 4236, 4815, 869, 491, 951], [510, 1775, 830, 3688, 191, 453, 4900, 1659, 4943, 834, 2500], [662, 1143, 985, 2266, 1256, 3849, 1241, 2059, 4943, 834, 2500], [4965, 4094, 1728, 729, 4320, 4581, 4997, 2092, 1241, 3318], [3145, 464, 4313, 2541, 1574, 3404, 3797, 2615], [144, 1500, 2439, 1210, 1306, 71, 1671, 1905, 755, 1174, 2972], [1194, 4951, 1838, 4581, 1241, 1256], [866, 4136, 2146, 2684, 141, 4596, 2270, 978, 1341], [1135, 1766, 3311, 1914, 2808, 3603, 3095, 4943, 834, 2500], [2294, 3230, 558, 414, 2885, 4943, 834, 2500], [3957, 231, 3933, 1026, 4159, 211, 4934, 767, 2159, 1054], [3730, 3995, 4883], [3568, 3401, 4226, 4633, 1241, 4631, 3395, 951], [28, 1084, 622, 4566, 2219, 1265, 3138, 3586, 1857], [2388, 10

## Embedding Representation

In [28]:
## Converting words to vectors
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)

embedded_docs

array([[   0,    0,    0, ..., 1184, 2067, 2124],
       [   0,    0,    0, ...,  776,  633,  951],
       [   0,    0,    0, ..., 2481,  517,  522],
       ...,
       [   0,    0,    0, ..., 4943,  834, 2500],
       [   0,    0,    0, ..., 3948,  429,  879],
       [   0,    0,    0, ..., 1717, 3084, 3514]], dtype=int32)

In [30]:
## Creating Model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [31]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [32]:
## Trainn Test Split

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final, test_size=0.2, random_state=0)

In [33]:
## Model training
model.fit(X_train, y_train, validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
458/458 [==============================] - 25s 42ms/step - loss: 0.2511 - accuracy: 0.8856 - val_loss: 0.1847 - val_accuracy: 0.9212
Epoch 2/10
458/458 [==============================] - 18s 40ms/step - loss: 0.1360 - accuracy: 0.9474 - val_loss: 0.1782 - val_accuracy: 0.9295
Epoch 3/10
458/458 [==============================] - 21s 45ms/step - loss: 0.0909 - accuracy: 0.9653 - val_loss: 0.2110 - val_accuracy: 0.9253
Epoch 4/10
458/458 [==============================] - 20s 44ms/step - loss: 0.0617 - accuracy: 0.9786 - val_loss: 0.2126 - val_accuracy: 0.9248
Epoch 5/10
458/458 [==============================] - 18s 40ms/step - loss: 0.0395 - accuracy: 0.9862 - val_loss: 0.2997 - val_accuracy: 0.9292
Epoch 6/10
458/458 [==============================] - 18s 40ms/step - loss: 0.0240 - accuracy: 0.9923 - val_loss: 0.3877 - val_accuracy: 0.9264
Epoch 7/10
458/458 [==============================] - 21s 46ms/step - loss: 0.0186 - accuracy: 0.9947 - val_loss: 0.3674 - val_accuracy:

In [40]:
## Predictions
y_pred = model.predict(X_test)

115/115 [==============================] - 1s 10ms/step


In [41]:
y_pred = np.where(y_pred >= 0.5,1,0)

In [42]:
from sklearn.metrics import confusion_matrix,accuracy_score

confusion_matrix(y_test,y_pred)

array([[1896,  144],
       [ 167, 1450]])

In [43]:
print(accuracy_score(y_test,y_pred))

0.9149576155318567
